In [3]:
from google.colab import files
file = files.upload()

Saving labelled_train_set (1).csv to labelled_train_set (1).csv


In [4]:
from google.colab import files
file = files.upload()

Saving unlabelled_test1.csv to unlabelled_test1.csv


In [189]:
import pandas as pd
df_train = pd.read_csv("labelled_train_set (1).csv")

In [190]:
df_test = pd.read_csv("unlabelled_test1.csv")

In [191]:
x_test = df_test["News/Comment"]
a = df_test["ID"]

In [192]:
temp = df_train["Type"]
li = []
for i in temp:
  if(i == "FALSE") :
    li.append(0)
  elif(i == "HALF TRUE"):
    li.append(1)
  elif(i == "MOSTLY FALSE"):
    li.append(2)
  elif(i == "PARTLY FALSE"):
    li.append(3)
  elif(i == "MOSTLY TRUE"):
    li.append(4)

In [193]:
df_train["Type"] = li

In [194]:
df_train["Type"][907]
df_train.drop(907, inplace = True)

In [195]:
x_train = df_train["News/Comment"]
y_train = df_train["Type"]

In [197]:
x_train_list = []
for i in x_train :
  x_train_list.append(str(i))
x_test_list = []
for i in x_test :
  x_test_list.append(str(i))

In [198]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
x_train = tfidf_vectorizer.fit_transform(x_train_list)
x_test = tfidf_vectorizer.transform(x_test_list)

In [199]:
print(y_train.value_counts())

Type
0    1130
2     230
1     138
3      38
Name: count, dtype: int64


In [200]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()
x_train, y_train = smote.fit_resample(x_train, y_train)

In [334]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators = 700)
rfc.fit(x_train, y_train)

RandomForestClassifier(n_estimators=700)

In [336]:
from sklearn.svm import SVC
svc = SVC(C = 0.1, kernel = "rbf", gamma = 0.01)
svc.fit(x_train, y_train)

SVC(C=0.1, gamma=0.01)

In [335]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter = 1000, solver = "lbfgs")

In [337]:
from sklearn.ensemble import StackingClassifier
base_models = [("rfc", rfc), ("svc", svc)]
meta_learner = lr
stacking_classifier = StackingClassifier(estimators = base_models, final_estimator = meta_learner)
stacking_classifier.fit(x_train, y_train)

StackingClassifier(estimators=[('rfc',
                                RandomForestClassifier(n_estimators=700)),
                               ('svc', SVC(C=0.1, gamma=0.01))],
                   final_estimator=LogisticRegression(max_iter=1000))

In [338]:
y_test_pred = stacking_classifier.predict(x_test)

In [343]:
import pandas as pd
import numpy as np
prediction = pd.DataFrame({"ID" : a, "Prediction" : y_test_pred})
prediction.to_csv('Prediction.csv', index = False)

In [344]:
from google.colab import files
files.download("Prediction.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>